In [13]:
import pandas as pd
import mariadb
import sys
import os
from dotenv import load_dotenv


df = pd.read_csv(r"C:\Users\dell\Downloads\DATA\dataset.csv")

# Drop the specified columns

# Example filter: rows where 'app_name' contains 'game' (case-insensitive)
game_filter = df['app_name'].str.contains('game', case=False)

# List of specific app names you want to filter by
games_to_keep = [
    "Fallout: New Vegas",
  
]

# Filter df by the specified game names
df = df[df['app_name'].isin(games_to_keep)]

# Limit to 60,000 rows
df = df.head(60000)

# Print value counts of 'app_name'
print(df['app_name'].value_counts())

# Convert to string and handle missing values
df['app_name'] = df['app_name'].astype(str)
df['review_text'] = df['review_text'].astype(str)
df['review_text'] = df['review_text'].fillna('')


db_host = os.getenv("DB_HOST"),
db_user = os.getenv("DB_USER"),
db_password = os.getenv("DB_PASSWORD"),
db_name = os.getenv("DB_NAME"),
try:
    # Establish a connection to the MariaDB server
    conn = mariadb.connect(
        user=db_user,
        password=db_password,
        host=db_host,
        database=db_name
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB: {e}")
    sys.exit(1)

cur = conn.cursor()

# Prepare the insert query with placeholders
insert_query = "INSERT INTO comments (game_name, review_text, review_score) VALUES (%s, %s, %s)"

# Extract data as a list of tuples
data_to_insert = list(df[['app_name', 'review_text', 'review_score']].itertuples(index=False, name=None))

try:
    # Execute many inserts at once
    cur.executemany(insert_query, data_to_insert)
    conn.commit()
    print("Insert successful!")
except mariadb.Error as e:
    print(f"Error inserting data: {e}")
    conn.rollback()

cur.close()
conn.close()

Fallout: New Vegas    32963
Name: app_name, dtype: int64
Insert successful!
